In [ ]:
import numpy as np
import matplotlib.pylab as plt
from dataset import Dataset
from soft_dtw import SoftDTW
from soft_dtw import jacobian_sq_euc
import numpy as np
from scipy.optimize import minimize


def softdtw_barycenter(X_list, Z_init, gamma=1.0, max_iter=50, tol=1e-3):
    """
    Compute the Soft-DTW barycenter of a list of time series.
    Reformulated version compatible with the user's SoftDTW class.
    """

    # correction : X_list is the correct name
    if X_list is None or len(X_list) == 0:
        raise ValueError("X_list must contain at least one time series.")

    weights = np.ones(len(X_list))  # uniform weights

    T, d = Z_init.shape

    def objective(Z_flat):
        Z = Z_flat.reshape(T, d)
        total_cost = 0.0
        total_grad = np.zeros_like(Z)

        for w, X in zip(weights, X_list):

            # Step 1: Distance matrix Z vs X
            D = SoftDTW.squared_euclidean_distances(None, Z, X)

            # Step 2: Soft-DTW forward + backward
            sdtw = SoftDTW(D, gamma=gamma)
            cost = sdtw.compute()
            E = sdtw._soft_dtw_grad()

            # Step 3: Gradient wrt Z
            G = jacobian_sq_euc(None, Z, X, E)

            total_cost += w * cost
            total_grad += w * G

        return total_cost, total_grad.ravel()

    # Optimization
    res = minimize(
        fun=objective,
        x0=Z_init.ravel(),
        method="L-BFGS-B",
        jac=True,
        tol=tol,
        options={"maxiter": max_iter, "disp": False},
    )

    return res.x.reshape(T, d)



def softdtw_kmeans(X_list, K, gamma=1.0, max_outer=30, max_inner=100, seed=0):
    """
    Soft-DTW K-means clustering compatible avec la classe SoftDTW personnalisée.
    X_list : liste de séries temporelles (np.array (T,d))
    """
    rng = np.random.RandomState(seed)

    # 1 — initialisation des barycentres par tirage de K séries
    indices = rng.choice(len(X_list), size=K, replace=False)
    Z = [X_list[idx].copy() for idx in indices]

    # initialisation des clusters
    assignments = np.zeros(len(X_list), dtype=int)

    for it in range(max_outer):
        print(f"[Iteration {it+1}/{max_outer}] Assignment step...")

        # ----------------------------
        # 2 — ASSIGNMENT
        # ----------------------------
        for i, X in enumerate(X_list):

            # coût pour chaque cluster
            costs = []
            for j in range(K):
                D = SoftDTW.squared_euclidean_distances(None, X, Z[j])
                sdtw = SoftDTW(D, gamma=gamma)
                costs.append(sdtw.compute())

            assignments[i] = np.argmin(costs)

        print(f"Recomputing barycenters...")

        # ----------------------------
        # 3 — UPDATE (barycentres)
        # ----------------------------
        new_Z = []
        for k in range(K):
            cluster_series = [X_list[i] for i in range(len(X_list)) if assignments[i] == k]

            if len(cluster_series) == 0:
                # cluster vide → réinitialiser
                new_Z.append(Z[rng.randint(K)])
                continue

            # initialisation barycentre = moyenne
            Z0 = np.mean(cluster_series, axis=0)

            # barycentre Soft-DTW
            Zk = softdtw_barycenter(
                X_list=cluster_series,
                Z_init=Z0,
                gamma=gamma,
                max_iter=max_inner
            )

            new_Z.append(Zk)

        Z = new_Z

    return assignments, Z


ImportError: cannot import name 'jacobian_sq_euc' from 'soft_dtw' (/Users/ilan/Code/MVA/Geometry Data Analysis/soft_dtw.py)

In [5]:
import matplotlib.pyplot as plt

def plot_softdtw_clusters(X_list, assignments, Z, K):
    plt.figure(figsize=(6, 14))
    
    for k in range(K):
        ax = plt.subplot(K, 1, k+1)

        cluster_series = [X_list[i] for i in range(len(X_list)) if assignments[i] == k]

        # séries du cluster en gris
        for x in cluster_series:
            ax.plot(x.ravel(), c="gray", alpha=0.3)

        # barycentre en rouge
        ax.plot(Z[k].ravel(), c="red", linewidth=2.5)

        ax.set_title(f"Cluster {k+1} ({len(cluster_series)} points)")

    plt.tight_layout()
    plt.show()


In [6]:
ds = Dataset("CBF")
X_tr, y_tr, X_te, y_te = ds.load_ucr()

# Convertir en liste simple
X_list = [x for x in X_tr]

K = len(np.unique(y_tr))  # nombre de clusters = nombre de classes

assignments, Z = softdtw_kmeans(
    X_list=X_list,
    K=K,
    gamma=1.0,
    max_outer=30,
    max_inner=100,
    seed=0
)

plot_softdtw_clusters(X_list, assignments, Z, K)


Loading UCR dataset: CBF
Path train: /Users/ilan/sdtw_data/UCR_TS_Archive_2015/CBF/CBF_TRAIN
Path test: /Users/ilan/sdtw_data/UCR_TS_Archive_2015/CBF/CBF_TEST
[Iteration 1/30] Assignment step...
Recomputing barycenters...


NameError: name 'jacobian_sq_euc' is not defined